In [1]:
import cv2
import depthai as dai
import numpy as np

import time
from pathlib import Path
from matplotlib import pyplot as plt

In [15]:
# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a source - left grayscale cameras
cam_left = pipeline.createMonoCamera()
cam_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

# Create outputs
xout_left = pipeline.createXLinkOut()
xout_left.setStreamName('left')
cam_left.out.link(xout_left.input)

In [21]:
## RECORDING FOR 10 SECONDS
## THE FRAMES ARE BEING STORED IN DIRECTORY "video_frames"

# Connect and start the pipeline
with dai.Device(pipeline) as device:

    # Output queue will be used to get the grayscale frames from the output defined above
    q = device.getOutputQueue(name="left", maxSize=4, blocking=False)

    # Make sure the destination path is present before starting to store the examples
    Path(f"video_frames/").mkdir(parents=True, exist_ok=True)

    # running loop for 10 secs
    ten_secs = time.time() + 10
    
    while time.time() < ten_secs:
        # Blocking call, will wait until a new data has arrived
        inSrc = q.get()  
        # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
        frame = inSrc.getCvFrame()
        # Frame is transformed and ready to be shown
        cv2.imshow("left", frame)
        cv2.waitKey(1)

        cv2.imwrite(f"video_frames/{int(time.time() * 10000)}.png", frame)

    cv2.destroyAllWindows()            

## Harris corner detection

In [22]:
# choosing a random image from the captured frames
image = cv2.imread('video_frames/16679490745743.png')

# display input image
cv2.imshow('Image input', image)
cv2.waitKey(1000)
cv2.destroyAllWindows()            

# converting to gray-scale
bw_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
bw_image = np.float32(bw_image)

# applying harris corner detection algorithm
dest = cv2.cornerHarris(bw_image, 2, 5, 0.07)
 
# Results are marked through the dilated corners
dest = cv2.dilate(dest, None)
 
# Reverting back to the original image,
# with optimal threshold value
image[dest > 0.01 * dest.max()]=[0, 0, 255]
 
# Image with corners marked 
# The image is stored in ______ for future references
cv2.imshow('Image with Borders', image)
cv2.waitKey(1000)
cv2.destroyAllWindows()

## Canny edge detection

In [23]:
# choosing a random image from the captured frames
image = cv2.imread('video_frames/16679490745743.png')

# display input image
cv2.imshow('Image original', image)
cv2.waitKey(1000)
cv2.destroyAllWindows()

# applying canny edge detection algorithm
edges_img = cv2.Canny(image,100,200)

# Image with edges marked 
# The image is stored in ______ for future references
cv2.imshow('Image with Borders', edges_img)
cv2.waitKey(1000)
cv2.destroyAllWindows()


## Homography

In [24]:
img1 = cv2.imread('video_frames/16679490734072.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

img2 = cv2.imread('video_frames/16679490775732.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [25]:
sift = cv2.SIFT_create() 

kp_img1, desc_img1 = sift.detectAndCompute(img1, None) 
kp_img2, desc_img2 = sift.detectAndCompute(img2, None) 

bf = cv2.BFMatcher()
matches = bf.knnMatch(desc_img1, desc_img2, k=2)

good_points=[]     
for m, n in matches: 
    if(m.distance < 0.6*n.distance): 
        good_points.append(m) 

In [27]:
query_pts = np.float32([kp_img1[m.queryIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
train_pts = np.float32([kp_img2[m.trainIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 
 
matches_mask = mask.ravel().tolist() 

h,w = img1.shape
 
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
 
dst = cv2.perspectiveTransform(pts, matrix)
 
homography = cv2.polylines(img2, [np.int32(dst)], True, (255, 0, 0), 3) 
 
cv2.imshow("Homography", homography) 
cv2.imshow("Img", img1) 
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [28]:
img3 = cv2.drawMatches(img1, kp_img1, img2, kp_img2, good_points, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("lines", img3)
cv2.waitKey(10000)
cv2.destroyAllWindows()